In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("How many users watched the same movie the same timestamp")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Ratings_Dataframe=(Ratings_Dataframe.withColumn("day",date_format('timestamp','dd'))
                                    .withColumn("hour",hour("timestamp"))
                  )
rdf=Ratings_Dataframe.groupBy("movieId","day",'hour').agg(count("userId").alias('NumberOfUsers'))
rdf.filter("NumberOfUsers>1").select(sum("NumberOfUsers")).show()
stagemetrics.end()

+------------------+
|sum(NumberOfUsers)|
+------------------+
|          18433506|
+------------------+



In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 5
numTasks => 214
elapsedTime => 90831 (1.5 min)
stageDuration => 89482 (1.5 min)
executorRunTime => 293454 (4.9 min)
executorCpuTime => 106653 (1.8 min)
executorDeserializeTime => 10870 (11 s)
executorDeserializeCpuTime => 2630 (3 s)
resultSerializationTime => 100 (0.1 s)
jvmGCTime => 5404 (5 s)
shuffleFetchWaitTime => 82 (82 ms)
shuffleWriteTime => 2497 (2 s)
resultSize => 839197 (819.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 1887436800
recordsRead => 40000528
bytesRead => 1381427650 (1317.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 9087140
shuffleTotalBlocksFetched => 1400
shuffleLocalBlocksFetched => 1400
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 145062144 (138.0 MB)
shuffleLocalBytesRead => 145062144 (138.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesR